## Import necessary libraries

In [6]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
import numpy as np
import pickle
import cv2 as cv
import tifffile as tiff
import skimage
from scipy import interpolate
from shapely.geometry import Polygon
from skimage.draw import polygon2mask
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
from scipy.ndimage import gaussian_filter1d
from tqdm import tqdm

## Load data
image_path and mesh_path need to be specified by the user.

In [9]:
# Load data (phase contrast images and mesh data)
# e.g. image_path = 'C:/SVM_Protocol/Images/MG1655_LB1/' and 
# mesh_path = 'C:/SVM_Protocol/Images/MG1655_LB1/MG1655_LB_meshdata.pkl'
image_path = 'path/to/images/'

mesh_path = 'path/to/_meshdata.pkl'

# Load in phase contrast images
files = [str(p) for p in Path(image_path).glob("*.tif")] 

imgs = []
for file in files:
    img = tiff.imread(file)
    imgs.append(img)
imgs = np.array(imgs)

with open(mesh_path, "rb") as file:
    # Load the data from the file using pickle.load()
    mesh_dataframe = pickle.load(file)

## Load functions

In [10]:
# If pixel size is not specified use default pixel size of 0.064.
# Micron per pixel
px = 0.064

# mesh2midline() takes two sets of mesh coordinates and calculates the midpoint coordinates between them. 
# The resulting midpoint coordinates are then smoothened using a spline to generate a midline.
def mesh2midline(x1, y1, x2, y2):
    x = (x1 + x2) / 2
    y = (y1 + y2) / 2
    line = np.array([x, y]).T
    line = smoothing_spline(line, n = len(x1), sf = 3, closed = False)
    return line

# smoothing_spline() smoothes a given contour using a spline interpolation. It takes the initial contour coordinates, 
# the number of points in the resulting smoothed contour, the smoothing factor, and whether the contour is closed or not.
def smoothing_spline(init_contour, n=200, sf=1, closed = True):
    if closed:
        tck, u = interpolate.splprep(init_contour.T, u=None, s=sf, per=1)
    else:
        tck, u = interpolate.splprep(init_contour.T, u=None, s=sf)
    u_new = np.linspace(u.min(), u.max(), n)
    x_new, y_new = interpolate.splev(u_new, tck, der=0)
    return np.array([x_new, y_new]).T

# get_inverted_image() takes an image, inverts it, and performs background subtraction to enhance the contrast of the inverted image. 
# It returns the inverted image and an interpolation object for further analysis.
def get_inverted_image(image):
    inverted_image = np.subtract(np.max(image),image)
    bgr = phase_background(inverted_image)
    inverted_image = np.maximum(inverted_image - bgr, 0)
    im_interp2d = interp2d(inverted_image)
    return inverted_image, im_interp2d

# phase_background() estimates the background intensity value of a phase contrast image by thresholding and morphological operations. 
# It returns the estimated background intensity value.
def phase_background(image, se_size=3, bgr_erode_num=1):
    cropped_image = image[int(image.shape[0]*0.05):int(image.shape[0]*0.95), int(image.shape[1]*0.05):int(image.shape[1]*0.95)]
    thres = skimage.filters.threshold_otsu(cropped_image)
    mask = cv.threshold(image, thres, 255, cv.THRESH_BINARY_INV)[1]
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (se_size, se_size))
    mask = cv.erode(mask, kernel, iterations=bgr_erode_num)
    bgr_mask = mask.astype(bool)
    bgr = np.mean(image[bgr_mask])
    return bgr

# interp2d() creates an interpolation object for a given image using the RectBivariateSpline function. 
# The interpolation object can be used to evaluate pixel values at non-grid locations.
def interp2d(image):
    ff = interpolate.RectBivariateSpline(range(image.shape[0]), range(image.shape[1]), image, kx=1, ky=1)
    return ff

# get_step_length() calculates the step lengths between consecutive points of two curves defined by coordinates x1, y1, x2, and y2. 
# The step lengths are multiplied by a pixel conversion factor px and returned.
def get_step_length(x1, y1, x2, y2, px):
    dx = x1[1:] + x2[1:] - x1[:-1] - x2[:-1]
    dy = y1[1:] + y2[1:] - y1[:-1] - y2[:-1]
    return (np.sqrt(dx**2 + dy**2) / 2)*px

# get_length() sums up all step_lengths to get the cell length.
def get_length(step_length):
    return np.sum(step_length)

# The cell width is computed as the mean of the top one-third of the widths. 
# It returns the average width and the individual widths.
def get_avg_width(x1, y1, x2, y2, px):
    width_not_ordered = np.sqrt((x1-x2)**2 + (y1-y2)**2)
    sorted_width = sorted(width_not_ordered, reverse=True)
    width = (sum(sorted_width[:len(sorted_width)//3]) / (len(sorted_width)//3))
    return width*px, width_not_ordered*px

# Calculate the cell area based on the contour that defines the cell.
def get_area(contour, px):
    poly = Polygon(contour)
    area = poly.area
    return area*px*px

# Calculate the volume using the trapezoidal rule.
def get_volume(x1, y1, x2, y2, step_length, px):
    d = np.sqrt((x1-x2)**2+(y1-y2)**2)
    volume = np.trapz((np.pi*(d/2)**2) , dx = step_length)
    return volume*px*px

# Calculates the surface area of a structure based on a series of widths and step lengths. 
# It computes the surface area for each width segment and returns the total surface area.
def get_surface_area(width_no, step_length):
    widths = width_no[1:]
    surface_areas = 2 * np.pi * (widths / 2) * step_length
    total_surface_area = np.sum(surface_areas)
    return total_surface_area

# Calculates various measurements related to the perimeter of a cell, 
# including the perimeter length, circularity, compactness, and sphericity.
def get_cell_perimeter_measurements(contour, area, px):
    v1 = contour[:-1]
    v2 = contour[1:]
    d = v2-v1
    perimeter = np.sum(np.sqrt(np.sum(d**2,axis=1)))*px
    circularity = (4*np.pi*area)/(perimeter)**2
    compactness = (perimeter ** 2) / area
    sphericity = (np.pi * 1.5 * (perimeter / (2 * np.pi)) ** 1.5) / area
    return perimeter, circularity, compactness, sphericity

# Calculates the maximum, minimum, and mean curvature values along a contour. 
# It uses the gradient and second derivative of the contour coordinates to compute the curvature.
def get_curvature_characteristics(contour, px):
    dx = np.gradient(contour[:, 0]*px)
    dy = np.gradient(contour[:, 1]*px)
    d2x = np.gradient(dx)
    d2y = np.gradient(dy)
    curvature = np.abs(dx*d2y - dy*d2x) / (dx**2 + dy**2)**1.5
    max_c = np.max(curvature)
    min_c = np.min(curvature)
    mean_c = np.nanmean(curvature)
    return max_c, min_c, mean_c

# Calculates the total phase intensity within the contour mask
def get_total_phaco_intensity(contour, shape, interp2d):
    mask = polygon2mask(shape, contour)
    coords = np.column_stack(np.where(mask)).astype(int)
    values = interp2d.ev(coords[:,0], coords[:,1])
    return np.sum(values), np.max(values), np.mean(values)

# Calculates the intensity values along a contour by evaluating the pixel values using the interpolation object interp2d. 
def get_contour_intensity(contour, interp2d):
    data =  interp2d.ev(contour.T[0], contour.T[1])
    return data

# Identifies the number of peaks in a signal by applying peak detection using the find_peaks function. 
# It considers peaks with a certain prominence and height threshold based on the maximum value.
def find_signal_peaks(signal, maximum):
    peaks, _ = find_peaks(signal, prominence= 0.5, height = (maximum * 0.5))
    return len(peaks)

# Measures the variability of a signal along a contour.
# It calculates the standard deviation of a rolling window of the signal and returns the resulting variability values.
def measure_contour_variability(signal):
    extra_contour_intensities = np.concatenate([signal, signal[:10]])
    contour_intensities_variability = np.array([np.std(extra_contour_intensities[ss:ss+10]) for ss in range(1, len(extra_contour_intensities)-10)])
    return contour_intensities_variability

# Calculates the kurtosis of a signal, which measures the peakedness or flatness of the distribution.
def get_kurtosis(signal):
    return kurtosis(signal)

# Calculates the skewness of a signal, which measures the asymmetry of the distribution.
def get_skew(signal):
    return skew(signal)

# Generates a contour from two sets of mesh coordinates. 
# the function combines the coordinates, smoothes the contour using a spline, and returns the resulting contour.
def mesh2contour(x1, y1, x2, y2):
    x2f = np.flip(x2)
    y2f = np.flip(y2)
    # Concatenate the x and y coordinates
    xspline = np.concatenate((x2f[1:], x1[1:]))
    yspline = np.concatenate((y2f[1:], y1[1:]))

    tck, u = interpolate.splprep(np.array([xspline, yspline]), k=3, s=2, per = 1)
    u_new = np.linspace(u.min(), u.max(), 200)
    outx, outy = interpolate.splev(u_new, tck)
    
    return np.array([outx, outy]).T

# Measures intensity values along a midline by sampling the image using an interpolation object im_interp2d. 
# The width and subpixel parameters control the width of the sampling region and the subpixel precision, respectively. 
# It returns the intensity profile along the midline.
def measure_along_midline_interp2d(midline, im_interp2d, width = 7, subpixel = 0.5):

    unit_dxy = unit_perpendicular_vector(midline, closed = False)
    width_normalized_dxy = unit_dxy * subpixel
    
    data = im_interp2d.ev(midline.T[0], midline.T[1])
    for i in range(1, 1+int(width * 0.5 / subpixel)):
        dxy = width_normalized_dxy * i
        v1 = midline + dxy
        v2 = midline - dxy
        p1 = im_interp2d.ev(v1.T[0], v1.T[1])
        p2 = im_interp2d.ev(v2.T[0], v2.T[1])
        data = np.vstack([p1, data, p2])
    prf = np.average(data, axis=0)
    sigma = 2  # standard deviation of Gaussian filter
    prf = gaussian_filter1d(prf, sigma)
    return prf

# Calculates the unit perpendicular vector to a set of coordinates. 
# The function computes the vector based on the angle between consecutive points.
def unit_perpendicular_vector(data, closed= True):

    p1 = data[1:]
    p2 = data[:-1]
    dxy = p1 - p2
    ang = np.arctan2(dxy.T[1], dxy.T[0]) + 0.5 * np.pi
    dx, dy = np.cos(ang), np.sin(ang)
    unit_dxy = np.array([dx, dy]).T
    if not closed:
        unit_dxy = np.concatenate([[unit_dxy[0]], unit_dxy])
    else:
        unit_dxy = np.concatenate([unit_dxy,[unit_dxy[-1]]])
    return unit_dxy

# Computes the coordinates of the expanded contour
def expand_contour(contour, scale= 1):
    
    area = 0.5 * np.sum(np.diff(contour[:, 0]) * (contour[:-1, 1] + contour[1:, 1]))
    if area < 0:
        dxy = unit_perpendicular_vector(contour, closed=True) 
    else:
        dxy = unit_perpendicular_vector(contour, closed=True) * (-1)

    return contour - (scale * dxy)

# Computes the coordinates for an eroded contour
def erode_contour(contour, scale= 1):

    area = 0.5 * np.sum(np.diff(contour[:, 0]) * (contour[:-1, 1] + contour[1:, 1]))
    if area < 0:
        dxy = unit_perpendicular_vector(contour, closed=True) 
    else:
        dxy = unit_perpendicular_vector(contour, closed=True) * (-1)

    return contour + (scale * dxy)

## Extract features

In [ ]:
# Add a column 'contour' to store the array of coordinates that make up the contour. 
mesh_dataframe['contour'] = None
# Add a column named 'discard'. True means the cell will be discarded, False indicates the cell will be kept for further analysis.
mesh_dataframe['discard'] = False
# Loop over all frames
for frame_num in tqdm(np.unique(mesh_dataframe['frame'])):
    frame_df = mesh_dataframe[mesh_dataframe['frame'] == frame_num]
    image = imgs[frame_num]
    # Get the inverted image and 2D interpolation
    inverted_image, im_interp2d = get_inverted_image(image)
    # Loop over all cells in the DataFrame
    for index, row in frame_df.iterrows():
        
        cell_id = row['cell_id']
        mesh = row['mesh']
        # Separate mesh coordinates
        x1 = mesh[:,0]
        y1 = mesh[:,1]
        x2 = mesh[:,2]
        y2 = mesh[:,3]
        # Discard meshes that are too small or large (4 <= not discarded <= 500)
        if 4 <= len(x1) <= 500: 
            try:
                step_length = get_step_length(x1, y1, x2, y2, px)
                   
                # Contour is added here for labeling purposes
                contour = mesh2contour(x1, y1, x2, y2)
               
                mesh_dataframe.at[index, 'contour'] = contour
                   
                # Morphological features
                mesh_dataframe.loc[index, 'cell_length'] = get_length(step_length)
                mesh_dataframe.loc[index, 'cell_width'], width_not_ordered = get_avg_width(x1, y1, x2, y2, px)
                cell_area = get_area(contour, px)
                mesh_dataframe.loc[index, 'cell_area'] = cell_area
                mesh_dataframe.loc[index, 'cell_volume'] = get_volume(x1, y1, x2, y2, step_length, px)
                mesh_dataframe.loc[index, 'cell_surface_area'] = get_surface_area(width_not_ordered, step_length)
                mesh_dataframe.loc[index, 'max_curvature'], mesh_dataframe.loc[index, 'min_curvature'], mesh_dataframe.loc[index, 'mean_curvature'] = get_curvature_characteristics(contour, px)
                mesh_dataframe.loc[index, 'cell_perimeter'], mesh_dataframe.loc[index, 'cell_circularity'], mesh_dataframe.loc[index, 'cell_POA'], mesh_dataframe.loc[index, 'cell_sphericity'] = get_cell_perimeter_measurements(contour, cell_area, px)
                   
                # Cell surface intensity features
                total_int, max_int, mean_int = get_total_phaco_intensity(contour, image.shape, im_interp2d)
                mesh_dataframe.loc[index, 'phaco_total_intensity'] = total_int
                mesh_dataframe.loc[index, 'phaco_max_intensity'] = max_int
                mesh_dataframe.loc[index, 'phaco_mean_intensity'] = mean_int
                
                phaco_contour_intensity = get_contour_intensity(contour, im_interp2d)
                phaco_contour_intensity_variability = measure_contour_variability(phaco_contour_intensity)
                
                sorted_phaco_contour_intensity = sorted(phaco_contour_intensity)
                mesh_dataframe.loc[index, 'phaco_contour_peaks'] = find_signal_peaks(phaco_contour_intensity, max_int)
                mesh_dataframe.loc[index, 'phaco_max_contour_intensity'] = np.max(phaco_contour_intensity)
                mesh_dataframe.loc[index, 'phaco_mean_contour_intensity'] = np.mean(phaco_contour_intensity)
                mesh_dataframe.loc[index, 'phaco_min_contour_intensity'] = np.mean(sorted_phaco_contour_intensity[:10])
                mesh_dataframe.loc[index, 'phaco_max_contour_variability'] = np.max(phaco_contour_intensity_variability)
                mesh_dataframe.loc[index, 'phaco_mean_contour_variability'] = np.mean(phaco_contour_intensity_variability)
                
                # Midline intensity features
                midline = mesh2midline(x1, y1, x2, y2)
                axial_intensity = measure_along_midline_interp2d(midline, im_interp2d, width=5)
                
                mesh_dataframe.loc[index, 'midline_kurtosis'] = get_kurtosis(axial_intensity)
                mesh_dataframe.loc[index, 'midline_skewness'] = get_skew(axial_intensity)
                
                # Expanded contour intensity features
                expanded_contour = expand_contour(contour, scale=2)
                eroded_contour = erode_contour(contour, scale=2)
                phaco_expanded_contour_intensity = get_contour_intensity(expanded_contour, im_interp2d)
                phaco_eroded_contour_intensity = get_contour_intensity(eroded_contour, im_interp2d)
                
                mesh_dataframe.loc[index, 'phaco_max_expanded_contour_intensity'] = np.max(phaco_expanded_contour_intensity)
                mesh_dataframe.loc[index, 'phaco_mean_expanded_contour_intensity'] = np.mean(phaco_expanded_contour_intensity)
                
                # Mesh gradient features
                mesh_dataframe.loc[index, 'phaco_cell_edge_gradient'] = np.average(phaco_eroded_contour_intensity - phaco_expanded_contour_intensity)
            except ValueError:
                mesh_dataframe.loc[index, 'discard'] = True                
                print(f'Cannot extract features from mesh of Cell number {index}')
        else:
            mesh_dataframe.loc[index, 'discard'] = True
            print(f'The mesh of Cell number {index} is too small or large')

mesh_dataframe = mesh_dataframe.drop('mesh', axis=1)

# Display first 5 rows of the created DataFrame
mesh_dataframe.head()

## Save dataframe

In [ ]:
file_path = 'path/to/svm_features.pkl'
mesh_dataframe.to_pickle(file_path)